# Introduction
There are multiple reasons for analyzing a version control system like your Git repository. See for example Adam Tornhill's book ["Your Code as a Crime Scene"](https://pragprog.com/book/atcrime/your-code-as-a-crime-scene) or his upcoming book ["Software Design X-Rays"](http://www.adamtornhill.com/swevolution/reviewersprogress.html) for plenty of inspirations:

You can 
- analyze knowledge islands
- distinguish often changing code from stable code parts
- identify code that is temporal coupled to other code

Having the necessary data for those analyses in a [Pandas](http://pandas.pydata.org/) <tt>DataFrame</tt> gives you many possibilities to quickly gain insights into the evolution of your software system in various ways.

# The idea

In a preceding [blog post](https://www.feststelltaste.de/reading-a-git-log-file-output-with-pandas/), I showed you a way to read a Git log file with Pandas' DataFrame and [GitPython](https://gitpython.readthedocs.io/en/stable/). Looking back, this was really complicated and tedious. So, with a few tricks we can do it much better this time:

- We use GitPython's feature to directly access an underlying Git installation. This is way more faster than using GitPython's object representation of the commits and makes it possible to have everything we need in one notebook.
- We use in-memory reading by using StringIO to avoid unnecessary file access. This avoids storing the Git output on disk and read it from from disc again. This method is faster, too.
- We also exploit Pandas' <tt>read_csv</tt> method even more.  This makes the transformation of the Git log into a <tt>DataFrame</tt> as easy as pie.

# Exporting the Git repo's history
The first step is to connect GitPython with the Git repo. If we have an instance of the repo, we can gain access to the underlying Git installation of the operating system via <tt>repo.git</tt>.

In our case, we tap the [Spring PetClinic repo](https://github.com/spring-projects/spring-petclinic), a small sample application for the Spring framework (I also analyzed the huge [Linux repo](https://github.com/torvalds/linux/), works as well).

With the <tt>git_bin</tt>, we can execute almost any Git command we like directly. In our hypothetical use case, we want to retrieve some information about the change frequency of files. For this, we need the complete history of the Git repo including statistics for the changed files (via <tt>--numstat</tt>).

We use a little trick to make sure, that the format for the file's statistics fits nicely with the commit's metadata (SHA <tt>%h</tt>, UNIX timestamp <tt>%at</tt> and author's name <tt>%aN</tt>). The <tt>--numstat</tt> option provides data for additions and deletions for the affected file name in one line &ndash; separated by the tabulator character <tt>\t</tt>:  
<p>
<tt>1<b>\t</b>1<b>\t</b>some/file/name.ext</tt>
</p>

We use the same tabular separator <tt>\t</tt> for the format string:
<p>
<tt>%h<b>\t</b>%at<b>\t</b>%aN</tt>
</p>

And here is the trick: Additionally, we add the number of tabulators of the file's statistics plus an additional tabulator in front of the format string to pretend that there is an empty file statistics' information in front of each commit meta data string.

The results looks like this:

<p>
<tt>\t\t\t%h\t%at\t%aN</tt>
</p>

Note: If you want to export the Git log on the command line into a file, you need to use the horizontal tab <tt>%x0A</tt> as separator instead of <tt>\t</tt> in the format string. Otherwise, the trick doesn't work (I'll show the corresponding format string at the end of this article).


OK, let's executed the Git log export:

In [19]:
import git 
import pandas as pd

GIT_REPO_PATH = r'../../spring-petclinic/'
repo = git.Repo(GIT_REPO_PATH)
git_bin = repo.git
git_log = git_bin.execute('git log --pretty=format:"%h\t%at\t%aN\t%s"')
commits = pd.read_csv(StringIO(git_log), 
    sep="\t",
    header=None,              
    names=['sha', 'timestamp', 'author', 'message']
    )
commits['word_count'] = commits['message'].str.count(" ")+1
commits.head()

,sha,timestamp,author,message,word_count
0,101c9dc,1498817227,Dave Syer,Update Spring Boot and Thymeleaf versions,6
1,ffa967c,1492026060,Antoine Rey,spring-petclinic-angular1 repo renamed to spri...,5
2,fd1c742,1488784768,Antoine Rey,Do not fail maven build when git directing is ...,10
3,f792522,1487331121,Stephane Nicoll,Merge pull request #229 from henri-tremblay:eh...,6
4,75912a0,1487331057,Stephane Nicoll,Polish contribution,2


In [17]:
s = 


0        6
1        6
2        5
3        5
4       10
5       10
6        6
7        2
8        2
9        2
10       3
11       3
12       3
13       3
14       5
15       5
16       5
17       5
18       4
19       4
20       4
21       4
22       4
23       1
24       1
25       7
26       7
27       1
28       1
29       5
        ..
5049     6
5050     6
5051     6
5052     6
5053     6
5054     6
5055     6
5056     6
5057     6
5058     6
5059     6
5060     6
5061     6
5062     6
5063     6
5064     6
5065     6
5066     6
5067     6
5068     6
5069     6
5070     6
5071     6
5072     6
5073     6
5074     6
5075     6
5076     6
5077     2
5078     2
Name: message, Length: 5079, dtype: int64

# Reading the Git log
We now read in the complete files' history in the <tt>git_log</tt> variable. Don't let confuse you by all the <tt>\t</tt> characters. 

Let's read the result into a Pandas <tt>DataFrame</tt> by using the <tt>read_csv</tt> method. Because we can't provide a file path to a CSV data, we have to use StringIO to read in our in-memory buffered content.

Pandas will read the first line of the tabular-separated "file", sees the many tabular-separated columns and parses all other lines in the same format / column layout. Additionally, we set the <tt>header</tt> to <tt>None</tt> because we don't have one and provide nice names for all the columns that we read in.

In [7]:
import pandas as pd
from io import StringIO

commits_raw = pd.read_csv(StringIO(git_log), 
    sep="\t",
    header=None,              
    names=['additions', 'deletions', 'filename', 'sha', 'timestamp', 'author', "message"]
    )
commits_raw.head()

,additions,deletions,filename,sha,timestamp,author,message
0,NaN,NaN,NaN,101c9dc,1.498817e+09,Dave Syer,Update Spring Boot and Thymeleaf versions
1,2,3,pom.xml,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,ffa967c,1.492026e+09,Antoine Rey,spring-petclinic-angular1 repo renamed to spri...
3,1,1,readme.md,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,fd1c742,1.488785e+09,Antoine Rey,Do not fail maven build when git directing is ...


Now we have two different kinds of content for the rows:
- The commit meta data without file statistics (see rows with the indexes 0, 2 and 4 above)
- The file statistics without the commit meta data (see rows with the indexes 1 and 3 above)

But we are interested in the commit meta data for each file's statistic. For this, we forward fill (<tt>ffill</tt>) the empty commit meta data entries of the file statistics rows with the preceding commit's meta data via the <tt>DataFrame</tt>'s <tt>fillna</tt> method and <tt>join</tt> this data with the existing columns of the file statistics.

In [8]:
commits = commits_raw[['additions', 'deletions', 'filename']]\
            .join(commits_raw[['sha', 'timestamp', 'author', 'message']].fillna(method='ffill'))
commits.head()

,additions,deletions,filename,sha,timestamp,author,message
0,NaN,NaN,NaN,101c9dc,1.498817e+09,Dave Syer,Update Spring Boot and Thymeleaf versions
1,2,3,pom.xml,101c9dc,1.498817e+09,Dave Syer,Update Spring Boot and Thymeleaf versions
2,NaN,NaN,NaN,ffa967c,1.492026e+09,Antoine Rey,spring-petclinic-angular1 repo renamed to spri...
3,1,1,readme.md,ffa967c,1.492026e+09,Antoine Rey,spring-petclinic-angular1 repo renamed to spri...
4,NaN,NaN,NaN,fd1c742,1.488785e+09,Antoine Rey,Do not fail maven build when git directing is ...


This gives use the commit meta data for each file change!

Because we aren't interested in the pure commit meta data anymore, we drop all those rows that don't contain file statistics aka contain null values via <tt>dropna</tt>.

In [5]:
commits = commits.dropna()
commits.head()

,additions,deletions,filename,sha,timestamp,author
1,2,3,pom.xml,101c9dc,1.498817e+09,Dave Syer
3,1,1,readme.md,ffa967c,1.492026e+09,Antoine Rey
5,1,0,pom.xml,fd1c742,1.488785e+09,Antoine Rey
8,1,1,pom.xml,75912a0,1.487331e+09,Stephane Nicoll
9,11,9,src/main/java/org/springframework/samples/petc...,75912a0,1.487331e+09,Stephane Nicoll


And that's it! We are finished!

In summary, you just need a "one-liner" for converting the Git log file output that was exported with
```
git log --numstat --pretty=format:"%x09%x09%x09%h%x09%at%x09%aN" > git.log
```
and read into a <tt>DataFrame</tt>:

In [6]:
# reading
git_log = pd.read_csv(
    "../../spring-petclinic/git.log",
    sep="\t", 
    header=None,
    names=[
        'additions', 
        'deletions', 
        'filename', 
        'sha', 
        'timestamp', 
        'author'])

# converting in "one line"
git_log[['additions', 'deletions', 'filename']]\
    .join(git_log[['sha', 'timestamp', 'author']]\
    .fillna(method='ffill'))\
    .dropna().head()

,additions,deletions,filename,sha,timestamp,author
1,2,3,pom.xml,101c9dc,1.498817e+09,Dave Syer
3,1,1,readme.md,ffa967c,1.492026e+09,Antoine Rey
5,1,0,pom.xml,fd1c742,1.488785e+09,Antoine Rey
8,1,1,pom.xml,75912a0,1.487331e+09,Stephane Nicoll
9,11,9,src/main/java/org/springframework/samples/petc...,75912a0,1.487331e+09,Stephane Nicoll


# Summary
In this notebook, I showed you how you can read a Git log output in only one line by using Pandas' <tt>read_csv</tt> method. This is a very handy method and a good starting point for further analyses! 